In [1]:
import numpy as np
import pandas as pd

from ols_bootstrap.pairs import PairsBootstrap
from ols_bootstrap.residual import ResidualBootstrap
from ols_bootstrap.wild import WildBootstrap

pd.options.display.float_format = '{:20,.5f}'.format  ### Setting pd to have a numerical precision up to 5 decimal points

In [2]:
df = pd.read_csv('./balance2018.csv')
df = df[df['sales_clean'] != 0]
df = df[['sales_clean', 'tanass_clean', 'tax']]
df = df.dropna(subset=['tanass_clean', 'tax'])

df_scaled = df.applymap(lambda x: np.log(x + 1))

/home/phamv/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_sample = df_scaled.sample(n=10000, replace=False)

Y_data = pd.DataFrame(df_sample.iloc[:, 0])
X_data = pd.DataFrame(df_sample.iloc[:, 1:])

## Default SE on the original OLS is HC3, default CI on bootstrapped parameter is BC. 

That is by default se_type = 'hc3', ci_type = 'bc'.

In [4]:
psb = PairsBootstrap(Y_data, X_data, reps = 1000, se_type='hc3', ci_type = 'bc')  # se_type = 'hc3' and ci_type = 'bc' are default options for these arguments.
psb.fit()

In [5]:
rsb = ResidualBootstrap(Y_data, X_data, reps = 1000)
rsb.fit()

In [6]:
wb_stdn = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "standard_normal")
wb_stdn.fit()

In [7]:
wb_rad = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "rademacher")
wb_rad.fit()

In [8]:
wb_mam = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "mammen")
wb_mam.fit()

In [9]:
psb.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|                     Pairs Bootstrap results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI                      |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2624   |      6.2600      | 0.0024 |     0.0443    |    0.0451    |    -1.95     | [6.1731, 6.3530] |  0.1799 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1681   |      0.1681      | 0.0000 |     0.0051 

In [10]:
rsb.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|                    Residual Bootstrap results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI                    |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2624   |      6.2605      | 0.0019 |     0.0443    |    0.0363    |    18.04     | [6.1930, 6.3361] |  0.1431 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1681   |      0.1683      | 0.0002 |     0.0051 

In [11]:
wb_stdn.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|           Wild Bootstrap with Standard Normal results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI            |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2624   |      6.2634      | 0.0010 |     0.0443    |    0.0336    |    23.99     | [6.1946, 6.3279] |  0.1332 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1681   |      0.1680      | 0.0001 |     0.0051 

In [12]:
wb_rad.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|              Wild Bootstrap with Rademacher results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI              |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2624   |      6.2635      | 0.0011 |     0.0443    |    0.0345    |    22.01     | [6.1877, 6.3231] |  0.1354 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1681   |      0.1681      | 0.0000 |     0.0051 

In [13]:
wb_mam.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|                Wild Bootstrap with Mammen results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI                |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2624   |      6.2615      | 0.0009 |     0.0443    |    0.0353    |    20.20     | [6.1954, 6.3290] |  0.1336 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1681   |      0.1681      | 0.0000 |     0.0051 

## Some useful methods were implemented (let's use it on wb_mam object of Wild Bootstrap with Mammen)

- The common in the following three methods is that either a string (if one wishes to capture one variable or ci) or 1-D like array can be provided to 'which_var' and/or 'which_ci' (the latter if exists in that object class).

### get_ci() method

#### Vanila version is when only the actual CI was used with all independent variables

In [14]:
wb_mam.get_ci()

bc                     
                              lwb                  upb
const                     6.19540              6.32902
tanass_clean              0.16000              0.17522
tax                       0.50245              0.52827

#### However, any combination of ('bc', 'bca', 'percentile) CI types could be selected and any combinations of independent variables can be chosen with 'which_ci' and 'which_var' optional arguments, respectively.

Please note that if choosing 'bca' the calculation can take a while as it uses jacknife resampling for calculating the acceleration factor

In [15]:
wb_mam.get_ci(which_ci=['bc', 'percentile'], which_var=['tax', 'const'])

bc                                percentile  \
                       lwb                  upb                  lwb   
tax                0.50245              0.52827              0.50318   
const              6.19540              6.32902              6.19538   

                            
                       upb  
tax                0.52855  
const              6.32898

In [16]:
wb_mam.get_ci(which_ci=['bca', 'bc'], which_var='tanass_clean')

bc                                       bca  \
                              lwb                  upb                  lwb   
tanass_clean              0.16000              0.17522              0.16003   

                                   
                              upb  
tanass_clean              0.17536

In [17]:
wb_mam.get_ci(which_ci='bc', which_var='tanass_clean')

bc                     
                              lwb                  upb
tanass_clean              0.16000              0.17522

### get_all_se() method

The following SE-s are calculated: 
- bootstrapped - standard error of the bootstrapped parameters
- constant - model-based OLS Standard Errors, that is, constant variance is assumed 
- HC0, HC1, HC2, HC3, HC4, HC4m, HC5 - Heteroskedasticity-Consistent Standard Errors (HCE) using sandwich estimators 

#### Vanila version is when using all indepencdent variables. 

In [18]:
wb_mam.get_all_se()

,bootstrapped,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
const,0.03532,0.03485,0.04424,0.04425,0.04425,0.04426,0.04426,0.04427,0.04426
tanass_clean,0.00391,0.00388,0.00509,0.00509,0.00509,0.00510,0.00510,0.00510,0.00510
tax,0.00660,0.00650,0.00917,0.00917,0.00917,0.00917,0.00918,0.00917,0.00918


#### A subset of indepent variables can be chosen with 'which_var' argument to calculate the above-mentioned 9 SE-s

In [19]:
wb_mam.get_all_se(which_var=['tanass_clean', 'tax'])

,bootstrapped,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
tanass_clean,0.00391,0.00388,0.00509,0.00509,0.00509,0.00510,0.00510,0.00510,0.00510
tax,0.00660,0.00650,0.00917,0.00917,0.00917,0.00917,0.00918,0.00917,0.00918


In [20]:
wb_mam.get_all_se(which_var='tax')

,bootstrapped,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
tax,0.00660,0.00650,0.00917,0.00917,0.00917,0.00917,0.00918,0.00917,0.00918


### get_bootstrap_params() method

#### Vanila version: Returning a dataframe capturing the parameter estimate of ALL each independent variables in each (wild) bootstrap.

In [21]:
wb_mam.get_bootstrap_params()

,const,tanass_clean,tax
0,6.26423,0.16165,0.52156
1,6.21812,0.17412,0.51076
2,6.32727,0.16332,0.51012
3,6.30109,0.16373,0.51205
4,6.22898,0.16458,0.52742
...,...,...,...
995,6.27720,0.16850,0.51776
996,6.27036,0.16236,0.51853
997,6.33722,0.16121,0.51146
998,6.26574,0.17030,0.51374


#### As usual, the desired independent variable can be chosen with 'which_var' argument

In [22]:
wb_mam.get_bootstrap_params(which_var='tax')

,tax
0,0.52156
1,0.51076
2,0.51012
3,0.51205
4,0.52742
...,...
995,0.51776
996,0.51853
997,0.51146
998,0.51374


In [23]:
wb_mam.get_bootstrap_params(which_var=('const', 'tanass_clean'))

,const,tanass_clean
0,6.26423,0.16165
1,6.21812,0.17412
2,6.32727,0.16332
3,6.30109,0.16373
4,6.22898,0.16458
...,...,...
995,6.27720,0.16850
996,6.27036,0.16236
997,6.33722,0.16121
998,6.26574,0.17030


### bp_test() and white_test() methods

#### Vanila version: for bp_test() returning the robust version (proposed by Roger Koenker) of this test when the presence of the heteroscedasticity is examined on the residual of the OLS on the original sample data. For white_test(), there are no optional arguments

In [24]:
wb_mam.bp_test()

(433.77931363663856,
 6.397626205131954e-95,
 226.65647911547447,
 5.381050797166677e-97)

In [25]:
wb_mam.white_test()

(1591.2774553930476, 0.0, 378.2554794698955, 0.0)

#### The non-robust version of bp_test() can be achieved by setting bp_test(robust = False).

In [26]:
wb_mam.bp_test(robust = False)

(1065.200647851445,
 4.950148860200041e-232,
 226.65647911547336,
 5.381050797169735e-97)

At each test, the first value is the calculated LM-test stats, the second is the 'LM p-value' corresponding to the LM-test stats, the third value is the F-test stats and last one is the 'F p-value'.

As it can be seen the p-values are 0, so we can rejcet the null hypothesis of homoscedasticity, and accept the alternative hypothesis that the variance of the residuals is heteroscedastic.